## Python で始める機器分析データの解析とケモメトリックス  
### 8.6. 主成分回帰 (PCR)

In [ ]:
import numpy
import pandas
from matplotlib import pyplot

In [ ]:
data = pandas.read_csv("data3.csv", header=0, index_col=0).T
data.index = pandas.read_csv("prop3.csv", header=0, index_col=0).iloc[:, 2].values
data = ((data.T - data.T.mean()) / data.T.std()).T
from scipy.signal import savgol_filter
buff = data
buff = savgol_filter(buff, 7, 2, 0)
buff = savgol_filter(buff, 7, 2, 1)
buff = savgol_filter(buff, 7, 2, 0)
buff = savgol_filter(buff, 7, 2, 1)
data = pandas.DataFrame(buff, index=data.index, columns=data.columns)
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, train_size=0.6, random_state=8)

In [ ]:
from sklearn.decomposition import PCA
model = PCA().fit(train.values)
pyplot.plot(model.explained_variance_ratio_)
pyplot.show()

In [ ]:
model = PCA(n_components=10).fit(train.values)
train_score = model.transform(train.values)
test_score = model.transform(test.values)

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(train_score, train.index)
calibration = model.predict(train_score)
validation = model.predict(test_score)

In [ ]:
pyplot.figure(figsize=(4, 4))
pyplot.plot([data.index.min(), data.index.max()], [data.index.min(), data.index.max()], c="black")
pyplot.scatter(train.index, calibration, label="calibration")
pyplot.scatter(test.index, validation, label="validation")
pyplot.legend()
pyplot.show()
from sklearn.metrics import mean_squared_error, r2_score
print("RMSEC =", numpy.sqrt(mean_squared_error(train.index, calibration)))
print("R^2 =", r2_score(train.index, calibration))
print("")
print("RMSEV =", numpy.sqrt(mean_squared_error(test.index, validation)))
print("R^2 =", r2_score(test.index, validation))

(c) Shigeaki Morita